# Unsteady Navier-Stokes problem

We consider the unsteady Navier-Stokes problem defined in the domain $\Omega := {[0,1]}^2$ by the following system of PDEs and Dirichlet BCs: 
$$
\begin{cases}
  \frac{\partial \boldsymbol{u}}{\partial t} +
  ({\boldsymbol{u}}\cdot\boldsymbol{\nabla})\ \boldsymbol{u}
  -\dfrac{1}{\text{Re}}\boldsymbol{\Delta} \boldsymbol{u} + \boldsymbol{\nabla} p = 0, 
  \qquad&\text{in }\Omega\times(0,1), \\
  \boldsymbol{\nabla} \cdot \boldsymbol{u} = 0, 
  \qquad &\text{in }\Omega\times(0,1), \\
  \boldsymbol{u} = \boldsymbol{g}, &\text{on }\partial\Omega\times(0,1),\\
  \boldsymbol{u}(\cdot,t=0) = \boldsymbol{u}_0, &\text{in }\Omega. 
\end{cases}
$$

We aim at evaluating the stability and convergence properties of the different time-advancing schemes on an analytical solution. The forcing term $\boldsymbol{f}$ is given by
$$
  \boldsymbol{f} =
  \begin{bmatrix}
    -2 \cos{x}\sin{y} \left( \cos{(2t)} + \dfrac{1}{\text{Re}} \sin{(2t)} \right) \\
    2 \sin{x}\cos{y} \left( \cos{(2t)} + \dfrac{1}{\text{Re}} \sin{(2t)} \right)
  \end{bmatrix},
$$
and the corresponding exact solution $(\boldsymbol{u}_{\text{ex}}, p_{\text{ex}})$ is defined by

$$
  \boldsymbol{u}_{\text{ex}} =
  \begin{bmatrix}
    - \cos{x}\sin{y} \sin{(2t)} \\
    \sin{x}\cos{y} \sin{(2t)}
  \end{bmatrix},
  \qquad
  p_{\text{ex}} =
  - \frac{\cos{(2x)}+\cos{(2y)}}{4} \sin^2{(2t)}.
$$

We consider the following **time discretizations of the convective term**:

*   the **backward Euler explicit** scheme for the convective term:
$$
\begin{cases}
  \frac{\boldsymbol{u}_{n+1}-\boldsymbol{u}_{n}}{\delta t} 
  +(\boldsymbol{u}_{n}\cdot\boldsymbol{\nabla})\boldsymbol{u}_{n} -
  \frac{1}{\text{Re}}\Delta \boldsymbol{u}_{n+1} 
  + \boldsymbol{\nabla} p_{n+1} =
        \boldsymbol{f}_{n+1},\\
        \boldsymbol{\nabla}\cdot\boldsymbol{u}_{n+1} = 0, \\
        \boldsymbol{u}_{n+1} = \boldsymbol{g}_{n+1};
      \end{cases}
$$

*  the **Euler semi-implicit** scheme:
$$
\begin{cases}
  \frac{\boldsymbol{u}_{n+1}-\boldsymbol{u}_{n}}{\delta t} 
  +(\boldsymbol{u}_{n}\cdot\boldsymbol{\nabla})\boldsymbol{u}_{n+1} -
  \frac{1}{\text{Re}}\Delta \boldsymbol{u}_{n+1} 
  + \boldsymbol{\nabla} p_{n+1} =
        \boldsymbol{f}_{n+1},\\
        \boldsymbol{\nabla}\cdot\boldsymbol{u}_{n+1} = 0, \\
        \boldsymbol{u}_{n+1} = \boldsymbol{g}_{n+1}.
      \end{cases}
$$

## Numerical solution with FEniCS

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import numpy as np

*   Study the stability of each method solving the problem with a decreasing time step ($\delta t =\frac{1}{2},\,\frac{1}{4},\,\frac{1}{8},\,\frac{1}{16}$) and an increasing number of subdivisions for the space discretization ($n=4,\,8,\,16, \,32$);
*   Verify numerically the convergence rates for each method, using a uniform mesh ($n=10$) and solving the problem with a decreasing time step ($\Delta t = \frac{1}{10},\,\frac{1}{20},\,\frac{1}{40},\,\frac{1}{80},\,\frac{1}{160}$).

*   Compute the error for the velocity and the pressure using the norm of the spaces $L^\infty(0,1,H^1(\Omega))$ and $L^\infty(0,1,L^2(\Omega))$, respectively.

In [ ]:
# 1. mesh generation
n = 10
dt = 1/40
mesh = UnitSquareMesh(n, n, 'crossed')

# 2. finite element spaces
V = VectorElement('CG', mesh.ufl_cell(), 2)
Q = FiniteElement('CG', mesh.ufl_cell(), 1)
X = FunctionSpace(mesh, V*Q)

u_exact = Expression((
    '- cos(x[0]) * sin(x[1]) * sin(2 * t)',
    'sin(x[0]) * cos(x[1]) * sin(2 * t)'
), degree=2, t=0.0)
p_exact = Expression(
    '-0.25 * (cos(2*x[0]) + cos(2*x[1])) * sin(2 * t) * sin(2 * t)',
    degree=2, t=0.0)

Re = Constant(1)
f = Expression((
    '-2 * cos(x[0]) * sin(x[1]) * (cos(2 * t) + sin(2 * t) / Re)',
    '2 * sin(x[0]) * cos(x[1]) * (cos(2 * t) + sin(2 * t) / Re)'
), degree=2, t=0.0, Re=Re)

def boundary(x, on_boundary):
  return on_boundary

def origin(x, on_boundary):
  return near(x[0], 0) and near(x[1], 0)

bc = [DirichletBC(X.sub(0), u_exact, boundary),
      DirichletBC(X.sub(1), p_exact, origin, 'pointwise')]

# 3. problem definition
u, p = TrialFunctions(X)
v, q = TestFunctions(X)

x_old = Function(X)
u_old, p_old = split(x_old)

The **backward Euler** time scheme corresponds to the following discrete formulation:

In [ ]:
# backward Euler explicit discretization of convective term
a = (dot(u, v) / Constant(dt) + inner(grad(u), grad(v)) / Re - p * div(v) - div(u) * q) * dx
L = (dot(u_old, v) / Constant(dt) - dot(grad(u_old) * u_old, v) + dot(f, v)) * dx

The **Euler semi-implicit** scheme corresponds to  

In [ ]:
# backward Euler semi-implicit discretization of convective term
a = (dot(u, v) / Constant(dt) + dot(grad(u)*u_old, v) + inner(grad(u), grad(v))/Re - p * div(v) - div(u) * q) * dx
L = (dot(u_old, v) / Constant(dt) + dot(f, v)) * dx

Another alternative is the **Euler semi-explicit** method, that has a strict requirement on the time step to ensure stability. It corresponds to

In [ ]:
# backward Euler semi-explicit scheme (unstable)
#a = (dot(u, v) / Constant(dt) - p * div(v) - div(u) * q) * dx
#L = (dot(u_old, v) / Constant(dt) - inner(grad(u_old), grad(v)) / Re - dot(grad(u_old) * u_old, v) + dot(f, v)) * dx

In [ ]:
# 4. solution
x = Function(X)
x_exact = Expression(('u[0]', 'u[1]', 'p'), degree=2, u=u_exact, p=p_exact)

# we compute the interpolate to initialize the advance in time
x.interpolate(x_exact)

time = 0.0
max_eH1 = 0.0
max_eL2 = 0.0

while time < 1.0:
  time += dt
  f.t = time
  u_exact.t = time
  p_exact.t = time

  x_old.assign(x)
  solve(a == L, x, bc)
  
  u, p = x.split()
  eH1 = errornorm(u_exact, u, 'H1')
  eL2 = errornorm(p_exact, p, 'L2')
  #print('time={:.2} eH1={:.3e} eL2={:.3e}'.format(time, eH1, eL2))
  
  max_eH1 = max(max_eH1, eH1)
  max_eL2 = max(max_eL2, eL2)

print('error eH1={:.3e} eL2={:.3e}'.format(max_eH1, max_eL2))

error eH1=1.531e-03 eL2=1.650e-02


### Solution with the fully-implicit Euler method

Each time-step of the **implicit Euler** scheme corresponds to the nonlinear problem

$$
\begin{cases}
  \frac{\boldsymbol{u}_{n+1}-\boldsymbol{u}_{n}}{\delta t} 
  +(\boldsymbol{u}_{n+1}\cdot\boldsymbol{\nabla})\boldsymbol{u}_{n+1} -
  \frac{1}{\text{Re}}\Delta \boldsymbol{u}_{n+1} 
  + \boldsymbol{\nabla} p_{n+1} =
        \boldsymbol{f}_{n+1},\\
        \boldsymbol{\nabla}\cdot\boldsymbol{u}_{n+1} = 0, \\
        \boldsymbol{u}_{n+1} = \boldsymbol{g}_{n+1}.
      \end{cases}
$$

Therefore, a linearization scheme is required for computing the approximating solution. In what follows we consider the **Newton method**.

In [ ]:
def solve_linear(x, advection, Re, dt, f, u_old, bc):
  u, p = TrialFunctions(X)
  v, q = TestFunctions(X)

  a = (dot(u, v)/Constant(dt) + inner(grad(u), grad(v))/Re - p * div(v) - div(u) * q) * dx
  a += (dot(grad(u) * advection, v) + dot(grad(advection) * u, v)) * dx
  L = (dot(u_old, v)/Constant(dt) + dot(f, v)) * dx
  L += dot(grad(advection) * advection, v) * dx
  
  x = Function(X)
  solve(a == L, x, bc)
  return x.split()

In [ ]:
# 0. problem data
dt = 1/10
n = 10
Re = Constant(1)

u_exact = Expression((
    '- cos(x[0]) * sin(x[1]) * sin(2 * t)',
    'sin(x[0]) * cos(x[1]) * sin(2 * t)'
), degree=2, t=0.0)
p_exact = Expression(
    '-0.25 * (cos(2*x[0]) + cos(2*x[1])) * sin(2 * t) * sin(2 * t)',
    degree=2, t=0.0)

f = Expression((
    '-2 * cos(x[0]) * sin(x[1]) * (cos(2 * t) + sin(2 * t) / Re)',
    '2 * sin(x[0]) * cos(x[1]) * (cos(2 * t) + sin(2 * t) / Re)'
), degree=2, t=0.0, Re=Re)

# 1. mesh generation
mesh = UnitSquareMesh(n, n, 'crossed')

# 2. finite element spaces
V = VectorElement('CG', mesh.ufl_cell(), 2)
Q = FiniteElement('CG', mesh.ufl_cell(), 1)
X = FunctionSpace(mesh, V*Q)

def boundary(x, on_boundary):
  return on_boundary

def origin(x, on_boundary):
  return near(x[0], 0) and near(x[1], 0)

bc = [DirichletBC(X.sub(0), u_exact, boundary),
      DirichletBC(X.sub(1), p_exact, origin, 'pointwise')]

# 3. problem definition
u, p = TrialFunctions(X)
v, q = TestFunctions(X)

# 4. solution
x = Function(X)
x_exact = Expression(('u[0]', 'u[1]', 'p'), degree=2, u=u_exact, p=p_exact)
x.interpolate(x_exact)
u_old, p_old = x.split() 

time = 0.0
max_eH1 = 0.0
max_eL2 = 0.0

niter = 20
tolerance = 1e-8

while time < 1.0:
  time += dt
  f.t = time
  u_exact.t = time
  p_exact.t = time
  
  # as initial guess we take the solution at the previous time step
  uh_old, ph_old = u_old, p_old
  
  for i in range(niter):
    uh, ph = solve_linear(X, uh_old, Re, dt, f, u_old, bc)
  
    error = (errornorm(uh, uh_old, 'H1') / norm(uh, 'H1') +
             errornorm(ph, ph_old, 'L2') / norm(ph, 'L2'))
    #print('time={:.2} - step={}: {:.3e}'.format(time, i, error))
    uh_old, ph_old = uh, ph
    if error < tolerance:
      break
  
  eH1 = errornorm(u_exact, uh, 'H1')
  eL2 = errornorm(p_exact, ph, 'L2')
  print('time={:.2} eH1={:.3e} eL2={:.3e}'.format(time, eH1, eL2))
  
  max_eH1 = max(max_eH1, eH1)
  max_eL2 = max(max_eL2, eL2)
  u_old, p_old = uh, ph

# 5. compute and print errors
print()
print('error eH1={:.3e} eL2={:.3e}'.format(max_eH1, max_eL2))

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
time=0.1 eH1=6.772e-04 eL2=4.403e-03
time=0.2 eH1=1.758e-03 eL2=1.084e-02
time=0.3 eH1=2.833e-03 eL2=1.686e-02
time=0.4 eH1=3.805e-03 eL2=2.220e-02
time=0.5 eH1=4.626e-03 eL2=2.666e-02
time=0.6 eH1=5.264e-03 eL2=3.006e-02
time=0.7 eH1=5.691e-03 eL2=3.225e-02
time=0.8 eH1=5.892e-03 eL2=3.317e-02
time=0.9 eH1=5.858e-03 eL2=3.275e-02
time=1.0 eH1=5.591e-03 eL2=3.104e-02
time=1.1 eH1=5.100e-03 eL2=2.808e-02

error eH1=5.892e-03 eL2=3.317e-02


### Results



Space-time convergence results:

$$
\begin{array}{ccccc}
& (n_t, n_x=n_y) & (2, 4) & (4, 8)  & (8, 16) & (16, 32) \\
explicit & L^2-\rm{error} &2.44\cdot 10^{-1}& 1.62\cdot 10^{-1} & 8.35\cdot 10^{-2} & 4.22\cdot 10^{-2}\\
explicit & H^1-\rm{error} &2.84\cdot 10^{-2}& 1.42\cdot 10^{-2} & 7.35\cdot 10^{-3} & 3.69\cdot 10^{-3} \\
& & & & \\
& (n_t, n_x=n_y) & (2, 4) & (4, 8)  & (8, 16) & (16, 32) \\
semi-implicit & L^2-\rm{error} &2.44\cdot 10^{-1}& 1.13\cdot 10^{-1} & 5.37\cdot 10^{-2} & 2.60\cdot 10^{-2} \\
semi-implicit & H^1-\rm{error} &2.84\cdot 10^{-2}& 1.42\cdot 10^{-2} & 7.34\cdot 10^{-3} & 3.69\cdot 10^{-3} \\
& & & & \\
& (n_t, n_x=n_y) & (2, 4) & (4, 8)  & (8, 16) & (16, 32) \\
implicit & L^2-\rm{error} & & & & 2.07\cdot 10^{-2} \\
implicit & H^1-\rm{error} & & & & 3.69\cdot 10^{-3} \\
\end{array}
$$

Convergence in time (with fixed $n_x=n_y=10$): 

$$
\begin{array}{ccccc}
& n_t & 10 & 20 & 40 & 80 & 160 \\
explicit & L^2-\rm{error} &6.63\cdot 10^{-2}& 3.34\cdot 10^{-2} & 1.65\cdot 10^{-2} & 8.05\cdot 10^{-3} & 4.16\cdot 10^{-3}\\
explicit & H^1-\rm{error} &5.89\cdot 10^{-3}& 2.98\cdot 10^{-3} & 1.53\cdot 10^{-3} & 8.40\cdot 10^{-4} & 5.44\cdot 10^{-4} \\
& & & & \\
& n_t & 10 & 20 & 40 & 80 & 160 \\
semi-implicit & L^2-\rm{error} &4.22\cdot 10^{-2}& 2.03\cdot 10^{-2} & 9.85\cdot 10^{-3} & 5.26\cdot 10^{-3} & 2.91\cdot 10^{-3}\\
semi-implicit & H^1-\rm{error} &5.89\cdot 10^{-3}& 2.98\cdot 10^{-3} & 1.53\cdot 10^{-3} & 8.40\cdot 10^{-4} & 5.44\cdot 10^{-4} \\
& & & & \\
& n_t & 10 & 20 & 40 & 80 & 160 \\
implicit & L^2-\rm{error} & & & & 4.26\cdot 10^{-3} & 2.30\cdot 10^{-3}\\
implicit & H^1-\rm{error} & & & & 8.40\cdot 10^{-4} & 5.44\cdot 10^{-4}\\
\end{array}
$$

In [ ]:
def solver_explicit(n, dt, u_exact, p_exact, f, Re):
  mesh = UnitSquareMesh(n, n, 'crossed')

  V = VectorElement('CG', mesh.ufl_cell(), 2)
  Q = FiniteElement('CG', mesh.ufl_cell(), 1)
  X = FunctionSpace(mesh, V*Q)

  def boundary(x, on_boundary):
    return on_boundary

  def origin(x, on_boundary):
    return near(x[0], 0.0) and near(x[1], 0.0)

  bc = [DirichletBC(X.sub(0), u_exact, boundary),
        DirichletBC(X.sub(1), p_exact, origin, 'pointwise')]

  u, p = TrialFunctions(X)
  v, q = TestFunctions(X)

  x_old = Function(X)
  u_old, p_old = split(x_old)

  a = (dot(u, v)/Constant(dt) + inner(grad(u), grad(v))/Re - p*div(v) - div(u)*q)*dx
  L = (dot(u_old, v)/Constant(dt) + dot(f, v) - dot(grad(u_old)*u_old, v))*dx

  #x_exact = Expression(('u[0]', 'u[1]', 'p'), degree=2, u=u_exact, p=p_exact)
  #x.interpolate(x_exact)

  x = Function(X)
  t = 0.0
  errH1 = 0.0
  errL2 = 0.0

  while t<1.0:

    t += dt
    f.time = t
    u_exact.time = t
    p_exact.time = t

    x_old.assign(x)
    solve(a==L, x, bc)
    u, p = x.split()

    errH1 = max(errH1, errornorm(u_exact, u, 'H1'))
    errL2 = max(errL2, errornorm(p_exact, p, 'L2'))

  return errH1, errL2

In [ ]:
def solver_semimplicit(n, dt, u_exact, p_exact, f, Re):
  mesh = UnitSquareMesh(n, n, 'crossed')

  V = VectorElement('CG', mesh.ufl_cell(), 2)
  Q = FiniteElement('CG', mesh.ufl_cell(), 1)
  X = FunctionSpace(mesh, V*Q)

  def boundary(x, on_boundary):
    return on_boundary

  def origin(x, on_boundary):
    return near(x[0], 0.0) and near(x[1], 0.0)

  bc = [DirichletBC(X.sub(0), u_exact, boundary),
        DirichletBC(X.sub(1), p_exact, origin, 'pointwise')]

  u, p = TrialFunctions(X)
  v, q = TestFunctions(X)

  x_old = Function(X)
  u_old, p_old = split(x_old)

  a = (dot(u, v)/Constant(dt) + dot(grad(u)*u_old, v) + inner(grad(u), grad(v))/Re - p*div(v) - div(u)*q)*dx
  L = (dot(u_old, v)/Constant(dt) + dot(f, v))*dx

  #x_exact = Expression(('u[0]', 'u[1]', 'p'), degree=2, u=u_exact, p=p_exact)
  #x.interpolate(x_exact)

  x = Function(X)
  t = 0.0
  errH1 = 0.0
  errL2 = 0.0

  while t<1.0:

    t += dt
    f.time = t
    u_exact.time = t
    p_exact.time = t

    x_old.assign(x)
    solve(a==L, x, bc)
    u, p = x.split()

    errH1 = max(errH1, errornorm(u_exact, u, 'H1'))
    errL2 = max(errL2, errornorm(p_exact, p, 'L2'))

  return errH1, errL2

In [ ]:
Re = Constant(1)

u_exact = Expression(('-cos(x[0])*sin(x[1])*sin(2*time)', 
                      'sin(x[0])*cos(x[1])*sin(2*time)'), time=0.0, degree=2)

p_exact = Expression('-0.25*(cos(2*x[0])+cos(2*x[1]))*sin(2*time)*sin(2*time)', time=0.0, degree=2)

# PAY ATTENTION: Below there's a * that should be a +
#p_exact = Expression('-0.25*(cos(2*x[0])*cos(2*x[1]))*sin(2*time)*sin(2*time)', time=0.0, degree=2)

f = Expression(('-2*cos(x[0])*sin(x[1])*(cos(2*time)+sin(2*time)/Re)',
                '2*sin(x[0])*cos(x[1])*(cos(2*time)+sin(2*time)/Re)'), time=0.0, Re=Re, degree=2)

for i in range(5):
  dt = 1/(10*(2**i))
  n = 10

  errH1, errL2 = solver_explicit(n, dt, u_exact, p_exact, f, Re)
  #errH1, errL2 = solver_semimplicit(n, dt, u_exact, p_exact, f, Re)
  print('dt={} errH1={:.3e} errL2={:.3e}'.format(dt, errH1, errL2))

dt=0.1 errH1=5.892e-03 errL2=4.216e-02
dt=0.05 errH1=2.979e-03 errL2=2.034e-02
dt=0.025 errH1=1.531e-03 errL2=9.854e-03
dt=0.0125 errH1=8.404e-04 errL2=5.265e-03
dt=0.00625 errH1=5.441e-04 errL2=2.913e-03


In [ ]:
def solver_implicit():

In [ ]:
dt = 1/20
n = 10
mesh = UnitSquareMesh(n, n, 'crossed')

V = VectorElement('CG', mesh.ufl_cell(), 2)
Q = FiniteElement('CG', mesh.ufl_cell(), 1)
X = FunctionSpace(mesh, V*Q)

def boundary(x, on_boundary):
  return on_boundary

def origin(x, on_boundary):
  return near(x[0], 0.0) and near(x[1], 0.0)

bc = [DirichletBC(X.sub(0), u_exact, boundary),
        DirichletBC(X.sub(1), p_exact, origin, 'pointwise')]

u, p = TrialFunctions(X)
v, q = TestFunctions(X)

x = Function(X)
x_old = Function(X)
u_old, p_old = split(x_old)

a = (dot(u, v)/Constant(dt) + inner(grad(u), grad(v))/Re - p*div(v) + div(u)*q)*dx
L = (dot(u_old, v)/Constant(dt) + dot(f, v) - dot(grad(u_old)*u_old, v))*dx

x_exact = Expression(('u[0]', 'u[1]', 'p'), degree=2, u=u_exact, p=p_exact)
x.interpolate(x_exact)

t = 0.0

errH1 = 0.0
errL2 = 0.0

while t<1.0:

  t += dt

  f.time = t
  u_exact.time = t
  p_exact.time = t

  x_old.assign(x)

  solve(a==L, x, bc)
  u, p = x.split()

  eH1 = errornorm(u_exact, u, 'H1')
  eL2 = errornorm(p_exact, p, 'L2')
  print('t={:.2} eH1={:.3e} eL2={:.3e}'.format(t, eH1, eL2))

t=0.05 eH1=4.018e-01 eL2=3.002e+00
t=0.1 eH1=1.010e-01 eL2=2.750e-02
t=0.15 eH1=2.705e-02 eL2=1.025e-02
t=0.2 eH1=6.753e-03 eL2=1.163e-02
t=0.25 eH1=1.084e-03 eL2=1.607e-02
t=0.3 eH1=1.009e-03 eL2=2.258e-02
t=0.35 eH1=1.618e-03 eL2=3.121e-02
t=0.4 eH1=1.955e-03 eL2=4.157e-02
t=0.45 eH1=2.195e-03 eL2=5.315e-02
t=0.5 eH1=2.392e-03 eL2=6.540e-02
t=0.55 eH1=2.560e-03 eL2=7.778e-02
t=0.6 eH1=2.700e-03 eL2=8.976e-02
t=0.65 eH1=2.813e-03 eL2=1.008e-01
t=0.7 eH1=2.897e-03 eL2=1.105e-01
t=0.75 eH1=2.953e-03 eL2=1.184e-01
t=0.8 eH1=2.979e-03 eL2=1.243e-01
t=0.85 eH1=2.975e-03 eL2=1.277e-01
t=0.9 eH1=2.942e-03 eL2=1.287e-01
t=0.95 eH1=2.879e-03 eL2=1.271e-01
t=1.0 eH1=2.787e-03 eL2=1.231e-01
